# 得点期待値をPythonで算出するサンプル

- 「ノーアウト一塁はバントがいいか打つべきか」みたいな課題を解く時に使う指標「得点期待値」を算出するコードのサンプルです.
- メジャーリーグのデータを使ってやっています.
- [Rによるセイバーメトリクス入門](https://gihyo.jp/book/2020/978-4-297-11684-2)の[サンプルコード](https://github.com/ghmagazine/r_analyzing_baseball/blob/main/chapter05.R)と比較すると良いかも.

In [1]:
# numpyとpandasを使います. 計算はほぼpandas
import csv
import numpy as np
import pandas as pd

## データ読み込みと下処理

- 事前に使うカラムを定義します
- データ型も付けちゃいます
- データは書籍同様, 2016年のメジャーリーグのデータを使います（プロ野球でやりたい人は自分でアレンジしてね）

In [2]:
# Dataframeで使うカラムとデータ型の定義
DF_COLUMNS = {
    'HOME_SCORE_CT': np.int32,
    'AWAY_SCORE_CT': np.int32,
    'GAME_ID': str,
    'INN_CT': np.int32,
    'BAT_HOME_ID': str,
    'BAT_DEST_ID': np.int32,
    'RUN1_DEST_ID': np.int32,
    'RUN2_DEST_ID': np.int32,
    'RUN3_DEST_ID': np.int32,
    'BASE1_RUN_ID': str,
    'BASE2_RUN_ID': str,
    'BASE3_RUN_ID': str,
    'OUTS_CT': np.int32,
    'EVENT_OUTS_CT': np.int32,
}

In [3]:
# CSVからDataframeを作る（headerとbody別れてるのでややこしいことになってます）
with open('./data/fields.csv', 'r') as f_header:
    headers = [row.get('Header') for row in csv.DictReader(f_header)]
with open('./data/all2016.csv', 'r') as f_body:
    values = [dict(zip(headers, row)) for row in csv.reader(f_body)]
df = pd.DataFrame(values, columns=DF_COLUMNS.keys()).astype(DF_COLUMNS)
df.head()

,HOME_SCORE_CT,AWAY_SCORE_CT,GAME_ID,INN_CT,BAT_HOME_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,OUTS_CT,EVENT_OUTS_CT
0,0,0,ANA201604040,1,0,2,0,0,0,,,,0,0
1,0,0,ANA201604040,1,0,0,0,3,0,,fowld001,,0,1
2,0,0,ANA201604040,1,0,0,0,0,3,,,fowld001,1,1
3,0,0,ANA201604040,1,0,1,0,0,4,,,fowld001,2,0
4,0,1,ANA201604040,1,0,0,1,0,0,rizza001,,,2,1


## いよいよ計算

- 書籍の方法に原則従っています
- 一部, RとPythonの違いを考慮しPythonicなやり方を取ってるところもあります.
- 最初は中間列を作るだるい作業が続きますご容赦ください.

In [4]:
# 得点を表すカラムを追加（ホームとアウェイで得点が別れてるので一緒にする）
df['RUNS'] = df['HOME_SCORE_CT'] + df['AWAY_SCORE_CT']


In [5]:
# イニングのKey値を表すカラム
df['HALF_INNING'] = df['GAME_ID'].astype(str) + df['INN_CT'].astype(str) + df['BAT_HOME_ID'].astype(str)


In [6]:
# 得点イベント時の総得点
def _run_scored(dests: str) -> int:
    """
    Calc Run Scored
    :param dests: Dests Strings
    :return: Run Scored
    """
    """
    Formatはこちら.DESTはすべて長さが1
    {BAT_DEST_ID}{RUN1_DEST_ID}{RUN2_DEST_ID}{RUN3_DEST_ID}
    """
    runs_scored = 0
    for dest in dests:
        if int(dest) > 3:
            runs_scored += 1
    return runs_scored
df['DESTS'] = df['BAT_DEST_ID'].astype(str) + df['RUN1_DEST_ID'].astype(str) + \
              df['RUN2_DEST_ID'].astype(str) + df['RUN3_DEST_ID'].astype(str)
df['RUNS_SCORED'] = df['DESTS'].map(_run_scored)

In [7]:
# ランナーの人数×アウトカウントでの状況を中間カラムとして設定
def _on_base(base_run_id) -> str:
    """
    Exists Runner
    :param base_run_id: retrosheet base_run_oid
    :return: '1' or '0'(1:True, 0:False)
    """
    if type(base_run_id) == float and math.isnan(base_run_id):
        return '0'
    elif type(base_run_id) == str and len(base_run_id) > 0:
        return '1'
    return '0'

df['BASES'] = df['BASE1_RUN_ID'].map(_on_base) + df['BASE2_RUN_ID'].map(_on_base) \
              + df['BASE3_RUN_ID'].map(_on_base)
df['STATE'] = df['BASES'].astype(str) + ' ' + df['OUTS_CT'].astype(str)


In [8]:
# プレー後の状況を表すカラム

def _new_base(dests: str) -> str:
    """
    Create New Base State
    :param dests: Dests
    :return: New Base
    """
    """
    Formatはこちら.DESTはすべて長さが1
    {BAT_DEST_ID}{RUN1_DEST_ID}{RUN2_DEST_ID}{RUN3_DEST_ID}_{func(OUTS_CT + EVENT_OUTS_CT)}
    """
    bat, run1, run2, run3 = int(dests[:1]), int(dests[1:2]), int(dests[2:3]), int(dests[3:4])
    nrunner1, nrunner2, nrunner3 = '0', '0', '0'
    if run1 == 1 or bat == 1:
        nrunner1 = '1'
    if run1 == 2 or run2 == 2 or bat == 2:
        nrunner2 = '1'
    if run1 == 3 or run2 == 3 or run3 == 3 or bat == 3:
        nrunner3 = '1'
    return f"{nrunner1}{nrunner2}{nrunner3}"

def _new_state(dest_outs: str) -> str:
    """
    Create New State
    :param dest_outs: Dests & Outs
    :return: New State
    """
    """
    Formatはこちら.DESTはすべて長さが1
    {BAT_DEST_ID}{RUN1_DEST_ID}{RUN2_DEST_ID}{RUN3_DEST_ID}_{func(OUTS_CT + EVENT_OUTS_CT)}
    """
    list_dest_outs = dest_outs.split('_')
    return f"{_new_base(list_dest_outs[0])} {list_dest_outs[1]}"


df['OUTS_CNT'] = df['OUTS_CT'] + df['EVENT_OUTS_CT']
df['DESTS_OUTS'] = df['DESTS'] + '_' + df['OUTS_CNT'].astype(str)
df['NEW_BASE'] = df['DESTS'].map(_new_base)
df['NEW_STATE'] = df['DESTS_OUTS'].map(_new_state)

In [9]:
# ここまでの途中経過
df.head()

,HOME_SCORE_CT,AWAY_SCORE_CT,GAME_ID,INN_CT,BAT_HOME_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,OUTS_CT,EVENT_OUTS_CT,RUNS,HALF_INNING,DESTS,RUNS_SCORED,BASES,STATE,OUTS_CNT,DESTS_OUTS,NEW_BASE,NEW_STATE
0,0,0,ANA201604040,1,0,2,0,0,0,,,,0,0,0,ANA20160404010,2000,0,000,000 0,0,2000_0,010,010 0
1,0,0,ANA201604040,1,0,0,0,3,0,,fowld001,,0,1,0,ANA20160404010,0030,0,010,010 0,1,0030_1,001,001 1
2,0,0,ANA201604040,1,0,0,0,0,3,,,fowld001,1,1,0,ANA20160404010,0003,0,001,001 1,2,0003_2,001,001 2
3,0,0,ANA201604040,1,0,1,0,0,4,,,fowld001,2,0,0,ANA20160404010,1004,1,001,001 2,2,1004_2,100,100 2
4,0,1,ANA201604040,1,0,0,1,0,0,rizza001,,,2,1,1,ANA20160404010,0100,0,100,100 2,3,0100_3,100,100 3


In [10]:
# 用が済んだカラムを捨てます
df.drop(columns=['DESTS', 'OUTS_CNT', 'DESTS_OUTS'], inplace=True)
df.head()

,HOME_SCORE_CT,AWAY_SCORE_CT,GAME_ID,INN_CT,BAT_HOME_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,OUTS_CT,EVENT_OUTS_CT,RUNS,HALF_INNING,RUNS_SCORED,BASES,STATE,NEW_BASE,NEW_STATE
0,0,0,ANA201604040,1,0,2,0,0,0,,,,0,0,0,ANA20160404010,0,000,000 0,010,010 0
1,0,0,ANA201604040,1,0,0,0,3,0,,fowld001,,0,1,0,ANA20160404010,0,010,010 0,001,001 1
2,0,0,ANA201604040,1,0,0,0,0,3,,,fowld001,1,1,0,ANA20160404010,0,001,001 1,001,001 2
3,0,0,ANA201604040,1,0,1,0,0,4,,,fowld001,2,0,0,ANA20160404010,1,001,001 2,100,100 2
4,0,1,ANA201604040,1,0,0,1,0,0,rizza001,,,2,1,1,ANA20160404010,0,100,100 2,100,100 3


## 算出する

- 得点状況を整理, 不要なカラムを消す
- 得点ROI（1得点までのコスト）を出す

In [11]:
# 中間結果としてイニングごとのruns_scoredを出す
df_runs_scored_inning = df[['RUNS_SCORED', 'HALF_INNING']].groupby('HALF_INNING', as_index=False).sum()
df_runs_scored_inning

,HALF_INNING,RUNS_SCORED
0,ANA20160404010,1
1,ANA20160404011,0
2,ANA20160404020,0
3,ANA20160404021,0
4,ANA20160404030,0
...,...,...
43415,WAS20161002070,0
43416,WAS20161002071,0
43417,WAS20161002080,2
43418,WAS20161002081,2


In [12]:
# さらに先頭の結果のみ抽出
df_runs_scored_start = df[['RUNS', 'HALF_INNING']].groupby('HALF_INNING', as_index=False).first()
df_runs_scored_start

,HALF_INNING,RUNS
0,ANA20160404010,0
1,ANA20160404011,1
2,ANA20160404020,1
3,ANA20160404021,1
4,ANA20160404030,1
...,...,...
43415,WAS20161002070,13
43416,WAS20161002071,13
43417,WAS20161002080,13
43418,WAS20161002081,15


In [13]:
# 上記の中間DataFrameをmerge
df_max = pd.merge(df_runs_scored_start, df_runs_scored_inning, how='inner', on='HALF_INNING')
df_max['MAX_RUNS'] = df_max['RUNS'] + df_max['RUNS_SCORED']
df_roi = pd.merge(df, df_max[['HALF_INNING', 'MAX_RUNS']], how='inner', on='HALF_INNING')
df_roi

,HOME_SCORE_CT,AWAY_SCORE_CT,GAME_ID,INN_CT,BAT_HOME_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,OUTS_CT,EVENT_OUTS_CT,RUNS,HALF_INNING,RUNS_SCORED,BASES,STATE,NEW_BASE,NEW_STATE,MAX_RUNS
0,0,0,ANA201604040,1,0,2,0,0,0,,,,0,0,0,ANA20160404010,0,000,000 0,010,010 0,1
1,0,0,ANA201604040,1,0,0,0,3,0,,fowld001,,0,1,0,ANA20160404010,0,010,010 0,001,001 1,1
2,0,0,ANA201604040,1,0,0,0,0,3,,,fowld001,1,1,0,ANA20160404010,0,001,001 1,001,001 2,1
3,0,0,ANA201604040,1,0,1,0,0,4,,,fowld001,2,0,0,ANA20160404010,1,001,001 2,100,100 2,1
4,0,1,ANA201604040,1,0,0,1,0,0,rizza001,,,2,1,1,ANA20160404010,0,100,100 2,100,100 3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190710,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,1,1,17,WAS20161002081,0,111,111 1,111,111 2,17
190711,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,2,1,17,WAS20161002081,0,111,111 2,111,111 3,17
190712,10,7,WAS201610020,9,0,1,0,0,0,,,,0,0,17,WAS20161002090,0,000,000 0,100,100 0,17
190713,10,7,WAS201610020,9,0,0,0,0,0,yelic001,,,0,2,17,WAS20161002090,0,100,100 0,000,000 2,17


In [14]:
# RUNS ROI算出
df_roi['RUNS_ROI'] = df_roi['MAX_RUNS'] - df_roi['RUNS']
df_roi

,HOME_SCORE_CT,AWAY_SCORE_CT,GAME_ID,INN_CT,BAT_HOME_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,OUTS_CT,EVENT_OUTS_CT,RUNS,HALF_INNING,RUNS_SCORED,BASES,STATE,NEW_BASE,NEW_STATE,MAX_RUNS,RUNS_ROI
0,0,0,ANA201604040,1,0,2,0,0,0,,,,0,0,0,ANA20160404010,0,000,000 0,010,010 0,1,1
1,0,0,ANA201604040,1,0,0,0,3,0,,fowld001,,0,1,0,ANA20160404010,0,010,010 0,001,001 1,1,1
2,0,0,ANA201604040,1,0,0,0,0,3,,,fowld001,1,1,0,ANA20160404010,0,001,001 1,001,001 2,1,1
3,0,0,ANA201604040,1,0,1,0,0,4,,,fowld001,2,0,0,ANA20160404010,1,001,001 2,100,100 2,1,1
4,0,1,ANA201604040,1,0,0,1,0,0,rizza001,,,2,1,1,ANA20160404010,0,100,100 2,100,100 3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190710,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,1,1,17,WAS20161002081,0,111,111 1,111,111 2,17,0
190711,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,2,1,17,WAS20161002081,0,111,111 2,111,111 3,17,0
190712,10,7,WAS201610020,9,0,1,0,0,0,,,,0,0,17,WAS20161002090,0,000,000 0,100,100 0,17,0
190713,10,7,WAS201610020,9,0,0,0,0,0,yelic001,,,0,2,17,WAS20161002090,0,100,100 0,000,000 2,17,0


In [15]:
# 必要なレコードのみに絞る. 具体的には「状況が変わった or 得点が入った」イベントのみ残す
df_roi.query("STATE != NEW_STATE or RUNS_SCORED > 0", inplace=True)
df_roi

,HOME_SCORE_CT,AWAY_SCORE_CT,GAME_ID,INN_CT,BAT_HOME_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,OUTS_CT,EVENT_OUTS_CT,RUNS,HALF_INNING,RUNS_SCORED,BASES,STATE,NEW_BASE,NEW_STATE,MAX_RUNS,RUNS_ROI
0,0,0,ANA201604040,1,0,2,0,0,0,,,,0,0,0,ANA20160404010,0,000,000 0,010,010 0,1,1
1,0,0,ANA201604040,1,0,0,0,3,0,,fowld001,,0,1,0,ANA20160404010,0,010,010 0,001,001 1,1,1
2,0,0,ANA201604040,1,0,0,0,0,3,,,fowld001,1,1,0,ANA20160404010,0,001,001 1,001,001 2,1,1
3,0,0,ANA201604040,1,0,1,0,0,4,,,fowld001,2,0,0,ANA20160404010,1,001,001 2,100,100 2,1,1
4,0,1,ANA201604040,1,0,0,1,0,0,rizza001,,,2,1,1,ANA20160404010,0,100,100 2,100,100 3,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190710,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,1,1,17,WAS20161002081,0,111,111 1,111,111 2,17,0
190711,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,2,1,17,WAS20161002081,0,111,111 2,111,111 3,17,0
190712,10,7,WAS201610020,9,0,1,0,0,0,,,,0,0,17,WAS20161002090,0,000,000 0,100,100 0,17,0
190713,10,7,WAS201610020,9,0,0,0,0,0,yelic001,,,0,2,17,WAS20161002090,0,100,100 0,000,000 2,17,0


In [16]:
# イニングごとにサマってアウトを合計, カラム名も変更
df_data_outs = df_roi[['HALF_INNING', 'EVENT_OUTS_CT']].groupby('HALF_INNING', as_index=False).sum()
df_data_outs.rename(columns={'EVENT_OUTS_CT': 'OutsInning'}, inplace=True)
df_data_outs

,HALF_INNING,OutsInning
0,ANA20160404010,3
1,ANA20160404011,3
2,ANA20160404020,3
3,ANA20160404021,3
4,ANA20160404030,3
...,...,...
43415,WAS20161002070,3
43416,WAS20161002071,3
43417,WAS20161002080,3
43418,WAS20161002081,3


In [17]:
# アウトになったデータをMerge

df_run_ex = pd.merge(df_roi, df_data_outs, how='inner', on='HALF_INNING')
df_run_ex

,HOME_SCORE_CT,AWAY_SCORE_CT,GAME_ID,INN_CT,BAT_HOME_ID,BAT_DEST_ID,RUN1_DEST_ID,RUN2_DEST_ID,RUN3_DEST_ID,BASE1_RUN_ID,BASE2_RUN_ID,BASE3_RUN_ID,OUTS_CT,EVENT_OUTS_CT,RUNS,HALF_INNING,RUNS_SCORED,BASES,STATE,NEW_BASE,NEW_STATE,MAX_RUNS,RUNS_ROI,OutsInning
0,0,0,ANA201604040,1,0,2,0,0,0,,,,0,0,0,ANA20160404010,0,000,000 0,010,010 0,1,1,3
1,0,0,ANA201604040,1,0,0,0,3,0,,fowld001,,0,1,0,ANA20160404010,0,010,010 0,001,001 1,1,1,3
2,0,0,ANA201604040,1,0,0,0,0,3,,,fowld001,1,1,0,ANA20160404010,0,001,001 1,001,001 2,1,1,3
3,0,0,ANA201604040,1,0,1,0,0,4,,,fowld001,2,0,0,ANA20160404010,1,001,001 2,100,100 2,1,1,3
4,0,1,ANA201604040,1,0,0,1,0,0,rizza001,,,2,1,1,ANA20160404010,0,100,100 2,100,100 3,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190551,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,1,1,17,WAS20161002081,0,111,111 1,111,111 2,17,0,3
190552,10,7,WAS201610020,8,1,0,1,2,3,harpb003,reveb001,turnt001,2,1,17,WAS20161002081,0,111,111 2,111,111 3,17,0,3
190553,10,7,WAS201610020,9,0,1,0,0,0,,,,0,0,17,WAS20161002090,0,000,000 0,100,100 0,17,0,3
190554,10,7,WAS201610020,9,0,0,0,0,0,yelic001,,,0,2,17,WAS20161002090,0,100,100 0,000,000 2,17,0,3


In [18]:
# 必要なものだけに絞ってデータ的には完成

df_run_ex = df_run_ex.query("OutsInning == 3")[['RUNS_ROI', 'STATE']].groupby('STATE', as_index=False).mean().round(2)
df_run_ex

,STATE,RUNS_ROI
0,000 0,0.50
1,000 1,0.27
2,000 2,0.11
3,001 0,1.35
4,001 1,0.94
...,...,...
19,110 1,0.92
20,110 2,0.41
21,111 0,2.11
22,111 1,1.54


## アウトプット用のデータにする

- 人様が読めるようにカラムを調整
- ピボットする

In [19]:
# ランナー状況の記号
RUNNER_STATUS_COLUMNS = {
    '000': {'key': '___', 'status': 'no runner'},  # ランナー無し
    '001': {'key': '__3', 'status': '3B'},  # 3塁
    '010': {'key': '_2_', 'status': '2B'},  # 2塁
    '011': {'key': '_23', 'status': '2B3B'},  # 2塁3塁
    '100': {'key': '1__', 'status': '1B'},  # 1塁
    '101': {'key': '1_3', 'status': '1B3B'},  # 1塁3塁
    '110': {'key': '12_', 'status': '1B2B'},  # 1塁2塁
    '111': {'key': '123', 'status': '1B2B3B'},  # 満塁
}

In [20]:
# 列を分割
df_runs_colums = df_run_ex['STATE'].str.split(' ', expand=True)
for key, value in RUNNER_STATUS_COLUMNS.items():
    df_runs_colums[0].replace(key, value.get('key'), inplace=True)
df_runs_colums

,0,1
0,___,0
1,___,1
2,___,2
3,__3,0
4,__3,1
...,...,...
19,12_,1
20,12_,2
21,123,0
22,123,1


In [21]:
# アウトおよび走者を再び代入
df_run_ex['outs'] = df_runs_colums[1]
df_run_ex['runner'] = df_runs_colums[0]
df_run_ex

,STATE,RUNS_ROI,outs,runner
0,000 0,0.50,0,___
1,000 1,0.27,1,___
2,000 2,0.11,2,___
3,001 0,1.35,0,__3
4,001 1,0.94,1,__3
...,...,...,...,...
19,110 1,0.92,1,12_
20,110 2,0.41,2,12_
21,111 0,2.11,0,123
22,111 1,1.54,1,123


In [22]:
# STATEはいらなくなったので消す
del df_run_ex['STATE']

In [23]:
# 表示用のDataFrameを作って完成
df_view = pd.pivot_table(df_run_ex, index='runner', columns='outs').reset_index()
df_view

runner RUNS_ROI            
outs               0     1     2
0       123     2.11  1.54  0.70
1       12_     1.44  0.92  0.41
2       1_3     1.72  1.20  0.48
3       1__     0.86  0.51  0.22
4       _23     1.93  1.36  0.55
5       _2_     1.13  0.67  0.31
6       __3     1.35  0.94  0.37
7       ___     0.50  0.27  0.11